doc2vec tutorial
====

1. 간단한 이론적 소개
2. 코퍼스 다운로드 및 전처리
3. 전통적인 방법
4. doc2vec 활용
5. 그때 그 방법 (word2vec)

1. 간단한 이론적 소개
----

### Distributed Representations of Sentences and Documents
* 논문: http://cs.stanford.edu/~quocle/paragraph_vector.pdf

### Bag-of-Words로 문서를 표현하는 방식의 문제점
* 단어의 순서를 잃어버림
  - bag-of-ngrams는 벡터의 차원을 기하급수적으로 높여 데이터 부족 문제를 심화시킴
* 단어의 의미를 담지 못함
  - 단어 간의 거리를 표현하지 못하고 모든 단어는 그냥 orthogonal한 자질로 표현됨

### Learning Vector Representation of Words
* Continuous Bag-of-Words (CBoW) Model
* Continuous Skip-Gram Model
* 기존 내용 참고: [word2vec tutorial](word2vec_tutorial.ipynb)

### Paragraph Vector
* Distributed Memory
  - considers the concatenation of the paragraph vector with the word vectors
  - to predict the next word in a text window
![](img/pv-dm.png)

* Distributed Bag-of-Words
  - to predict words randomly sampled from the paragraph in the output
  - similar to skip-gram model in word vectors
![](img/pv-dbow.png)

### Two Key Stages
* Training
  - to get word vectors W; softmax weights U, b; paragraph vectors D
* Inference
  - to get paragraph vectors D for new phragraphs
  - by adding more columns in D and gradient descending on D

### Experiments on Sentiment Analysis
* The Stanford Sentiment Treebank Dataset
  - single sentence
  - 11,855 (total) = 8,544 (train) + 2,210 (test) + 1,101 (develop)
  - 5-way, 2-way classification
![](img/pv-exp-stanford.png)

* IMDB Dataset
  - beyond sentence (paragraph/document)
  - 100,000 (total) = 25,000 (train) + 25,000 (test) + 50,000 (unlabeled)
  - 2-way {positive, negative}
![](img/pv-exp-imdb.png)

2. 코퍼스 다운로드 및 전처리
----

* Naver sentiment movie corpus v1.0
  - https://github.com/e9t/nsmc/
  - 참고: http://www.lucypark.kr/slides/2015-pyconkr


* 학습 집합

```
$ wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
```

* 평가 집합

```
$ wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
```

In [ ]:
### KoNLPy를 이용해 품사 태깅을 수행한다. => ratings_*.tag

from konlpy.tag import Mecab
from konlpy.utils import pprint
import os

mecab = Mecab('%s/usr/mecab-ko/lib/mecab/dic/mecab-ko-dic' % os.environ['HOME'])

def pos_tagging(in_path, out_path):
    """
    코퍼스를 읽어들여 품사태깅을 수행하는 함수
    @param  in_path   입력 파일 경로
    @param  out_path  출력 파일 경로
    """
    with open(out_path, 'w') as fout:
        for line_num, line in enumerate(open(in_path), start=1):
            if line_num == 1:    # 첫번째 줄(헤더)은 넘어갑니다.
                continue
            line = line.strip()
            if not line:
                continue
            doc_id, doc, rating = line.split('\t')
            tagged_doc = ['%s/%s' % (morph.encode('UTF-8'), tag.encode('UTF-8'))    # '형태소/태그' 형태로 저장합니다.
                          for morph, tag in mecab.pos(unicode(doc, 'UTF-8'))]
            # "문서ID  <tab>  문서(단어 목록)  <tab>  rating" 포맷으로 저장합니다.
            print >> fout, '%s\t%s\t%s' % (doc_id, ' '.join(tagged_doc), rating)

pos_tagging('ratings_train.txt', 'ratings_train.tag')
!head -n5 ratings_test.tag

pos_tagging('ratings_test.txt', 'ratings_test.tag')
!head -n5 ratings_test.tag

6270596	굳/VA ㅋ/UNKNOWN	1
9274899	GDNTOPCLASSINTHECLUB/SL	0
8544678	뭐/NP 야/VCP+EF 이/MM 평점/NNG 들/XSN 은/JX ./SF .../SY 나쁘/VA 진/EC+JX 않/VX 지만/EC 10/SN 점/NNBC 짜리/NNG 는/JX 더더욱/MAG 아니/VCN 잖아/EC	0
6825595	지루/XR 하/XSA 지/EC 는/JX 않/VX 은데/EC 완전/NNG 막장/NNG 임/VCP+ETN ./SF ../SY 돈/NNG 주/VV 고/EC 보/VX 기/ETN 에/JKB 는/JX ./SF .../SY	0
6723715	3/SN D/SL 만/JX 아니/VCN 었/EP 어도/EC 별/MM 다섯/NR 개/NNBC 줬/VV+EP 을/ETM 텐데/NNB+VCP+EF ./SF ./SY 왜/MAG 3/SN D/SL 로/JKB 나와서/VV+EC 제/MM 심기/NNG 를/JKO 불편/NNG 하/XSV 게/EC 하/VX 죠/EF ?/SF ?/SF	0
6270596	굳/VA ㅋ/UNKNOWN	1
9274899	GDNTOPCLASSINTHECLUB/SL	0
8544678	뭐/NP 야/VCP+EF 이/MM 평점/NNG 들/XSN 은/JX ./SF .../SY 나쁘/VA 진/EC+JX 않/VX 지만/EC 10/SN 점/NNBC 짜리/NNG 는/JX 더더욱/MAG 아니/VCN 잖아/EC	0
6825595	지루/XR 하/XSA 지/EC 는/JX 않/VX 은데/EC 완전/NNG 막장/NNG 임/VCP+ETN ./SF ../SY 돈/NNG 주/VV 고/EC 보/VX 기/ETN 에/JKB 는/JX ./SF .../SY	0
6723715	3/SN D/SL 만/JX 아니/VCN 었/EP 어도/EC 별/MM 다섯/NR 개/NNBC 줬/VV+EP 을/ETM 텐데/NNB+VCP+EF ./SF ./SY 왜/MAG 3/SN D/SL 로/JKB 나와서/VV+EC 제/MM 심기/NNG 를/JKO 불편/NNG 하/XSV 게/EC 하/VX 죠/EF ?/SF

In [ ]:
### 코퍼스를 읽어들입니다.

def load_corpus(path):
    """
    코퍼스를 읽어들이는 함수
    @param  path  경로
    @return       (doc_id, [words], rating) 튜플의 리스트 [(doc_id, [word1, word2, ...], rating), ...]
    """
    corpus = []
    for line in open(path):
        line = line.strip()
        if not line:
            continue
        doc_id, words, rating = line.split('\t')
        corpus.append((doc_id, [unicode(word, 'UTF-8') for word in words.split()],
                       1 if rating == '1' else -1))    # 긍정인 경우 1, 부정인 경우 -1
    return corpus

train_corpus = load_corpus('ratings_train.tag')
print train_corpus[0]

test_corpus = load_corpus('ratings_test.tag')
print test_corpus[0]

('9976970', [u'\uc544/IC', u'\ub354\ube59/NNG', u'./SF', u'./SY', u'\uc9c4\uc9dc/MAG', u'\uc9dc\uc99d/NNG', u'\ub098/NP', u'\ub124\uc694/VCP+EC', u'\ubaa9\uc18c\ub9ac/NNG'], -1)
('6270596', [u'\uad73/VA', u'\u314b/UNKNOWN'], 1)


3. 전통적인 방법
----

In [ ]:
### 전부 바이너리 벡터로 표현합니다.

import numpy as npy

def to_bin_vector(vocabulary, features):
    """
    vocabulary 사전을 이용하여 모든 자질을 하나의 bag-of-words 벡터로 표현하는 함수
    @param  vocabulary  vocabulary 사전
    @param  features    자질 목록
    @return             벡터
    """
    vec_sum = npy.zeros([len(vocabulary),], dtype=npy.int8)    # vocabulary 크기 만큼 벡터를 0으로 초기화 합니다.
    for feature in features:
        try:
            vec_sum[vocabulary[feature]] = 1    # 자질 번호에 해당하는 벡터 값을 1로 설정해 줍니다.
        except KeyError:
            pass
    return vec_sum

all_features = set()
for _, features, _ in train_corpus:    # 모든 자질을 합칩니다.
    all_features.update(features)
vocabulary = {feature: num for num, feature in enumerate(all_features)}    # 각 자질에 고유 번호를 부여하여 사전을 구축합니다.
print 'Number of vocabulary:', len(vocabulary)

# train 코퍼스를 전부 벡터로 표현합니다.
bin_train = [(rating, to_bin_vector(vocabulary, features)) for _, features, rating in train_corpus]
print 'First train vector:'
print bin_train[0]

# test 코퍼스를 전부 벡터로 표현합니다.
bin_test = [(rating, to_bin_vector(vocabulary, features)) for _, features, rating in test_corpus]
print 'First test vector:'
print bin_test[0]

Number of vocabulary: 62536
First train vector:
(-1, array([0, 0, 0, ..., 0, 0, 0], dtype=int8))
First test vector:
(1, array([0, 0, 0, ..., 0, 0, 0], dtype=int8))


In [ ]:
### Logistic Regression을 이용하여 학습 및 평가를 수행하는 함수를 작성합니다.

from sklearn import linear_model

def evaluate(train, test):
    """
    Logistic Regression을 이용하여 평가를 수행하는 함수
    @param  train  train 벡터 리스트
    @param  test   test 벡터 리스트
    @return        정확도
    """
    X_train = [feature for _, feature in train]
    y_train = [outcome for outcome, _ in train]
    X_test = [feature for _, feature in test]
    y_test = [outcome for outcome, _ in test]
    lin_model = linear_model.LogisticRegression()
    lin_model.fit(X_train, y_train)    # 학습 집합을 이용해 학습합니다.
    # 0 이상으로 예측한 경우 긍정, 아니면 부정으로 간주합니다.
    predicts = [1 if outcome >= 0 else -1 for outcome in lin_model.predict(X_test)]
    # 긍정/부정을 맞춘 경우 1점, 아니면 0점으로 모든 test 셋에 대해 평가합니다.
    corrects = [1.0 if predict == correct else 0.0 for predict, correct in zip(predicts, y_test)]
    return sum(corrects) / len(corrects)    # 정확도를 계산합니다.

In [ ]:
### 평가를 해봅니다.

evaluate(bin_train, bin_test)

# 오!

0.84946

![](https://pbs.twimg.com/profile_images/1194497929/goodmen_mark01.jpg)

4. doc2vec 활용
----

* doc2vec 관련 참고
  - https://radimrehurek.com/gensim/models/doc2vec.html
  - http://rare-technologies.com/doc2vec-tutorial/

In [ ]:
### gensim을 이용해 doc2vec을 만들어 봅시다. => ratings_train.d2v

import gensim

def documents():
    """
    train 코퍼스 전체를 읽어들여 문장(LabeledSentence 객체)을 하나씩 반환하는 제너레이터
    (리스트를 사용하여 메모리에 모두 로드하면 상당히 많은 메모리를 사용하게 됩니다.)
    """
    for doc_id, words, _ in train_corpus:
        # LabledSentence 객체는 words 멤버에 단어 리스트를, tags 멤버에 문서 ID 리스트를 주면 됩니다.
        yield gensim.models.doc2vec.LabeledSentence(words=words, tags=[doc_id,])

d2v_model = gensim.models.Doc2Vec(size=300, alpha=0.025, min_alpha=0.025, workers=20)
d2v_model.build_vocab(documents())    # vocabulary를 먼저 생성합니다.
for _ in range(10):    # 10회 반복하여 학습을 진행합니다.
    d2v_model.train(documents())
    d2v_model.alpha -= 0.002    # alpha 값을 줄여 learning rate를 서서히 줄입니다.
    d2v_model.min_alpha = d2v_model.alpha    # epoch 내에서는 learning rate를 decay 없이 고정합니다.
d2v_model.save('ratings_train.d2v')

In [ ]:
### doc2vec을 이용하여 벡터로 표현합니다.

import gensim

# train 코퍼스로 만든 doc2vec 모델을 읽어들입니다.
d2v_model = gensim.models.Doc2Vec.load('ratings_train.d2v')
print 'Vector size:', d2v_model.vector_size

# '추천/NNG'와 유사한 단어를 출력해 봅니다.
print 'Similar words with 추천/NNG:'
for word, sim in d2v_model.most_similar(u'추천/NNG'):
    print '    [%.4f] %s' % (sim, word.encode('UTF-8'))

# train 코퍼스를 전부 벡터로 표현합니다.
d2v_train = [(rating, d2v_model.infer_vector(features)) for _, features, rating in train_corpus]
print 'First train vector:'
print d2v_train[0][1][:30]

# test 코퍼스를 전부 벡터로 표현합니다.
d2v_test = [(rating, d2v_model.infer_vector(features)) for _, features, rating in test_corpus]
print 'First test vector:'
print d2v_test[0][1][:30]

Vector size: 300
Similar words with 추천/NNG:
    [0.6503] 강추/NNG
    [0.6335] 감사/NNG
    [0.5391] 응원/NNG
    [0.4978] 부탁/NNG
    [0.4796] 비추/NNG
    [0.4520] 후회/NNG
    [0.4327] 존경/NNG
    [0.4211] 만족/NNG
    [0.3727] 기원/NNG
    [0.3573] 인정/NNG
First train vector:
[ 0.01502489 -0.07158794  0.01497745 -0.01129568  0.00921065 -0.00502291
 -0.00182248 -0.08597456 -0.02611548 -0.00925563  0.02616826 -0.05886781
  0.06599369  0.01810654 -0.00656249  0.04815501  0.00165189  0.00895181
 -0.0426794   0.02889599  0.02255549 -0.03707462 -0.01186537 -0.0246332
  0.00721007 -0.03657341  0.05572002  0.01712912 -0.04992262 -0.02918133]
First test vector:
[  1.31950140e-01  -8.12067091e-03  -6.17027916e-02   3.71611528e-02
   4.56725769e-02   2.18428727e-02   1.29490504e-02  -3.02328803e-02
  -1.35810981e-02  -3.43034826e-02  -4.86127026e-02   9.03671421e-03
   1.72786862e-02  -5.15960827e-02  -8.57179761e-02   1.16714351e-02
  -5.10878675e-02   1.01754479e-01  -7.32302945e-03  -4.37404402e-02
  -3.94

In [ ]:
### 평가를 해봅니다.
evaluate(d2v_train, d2v_test)

# oTL

0.69524

![](https://pbs.twimg.com/profile_images/2104851627/image.jpg)

In [ ]:
### 댓글 코퍼스로부터 200만 댓글을 샘플링해 만든 doc2vec을 이용하여 벡터로 표현합니다.

import gensim

# 200만 댓글 코퍼스로 만든 doc2vec 모델을 읽어들입니다.
d2v_model = gensim.models.Doc2Vec.load('comment_corpus_2m.d2v')
print 'Vector size:', d2v_model.vector_size
for word, sim in d2v_model.most_similar(u'추천/NNG'):
    print '[%.4f] %s' % (sim, word.encode('UTF-8'))

# train 코퍼스를 전부 벡터로 표현합니다.
d2v_train = [(rating, d2v_model.infer_vector(features)) for _, features, rating in train_corpus]
print 'First train vector:'
print d2v_train[0][1][:30]

# test 코퍼스를 전부 벡터로 표현합니다.
d2v_test = [(rating, d2v_model.infer_vector(features)) for _, features, rating in test_corpus]
print 'First test vector:'
print d2v_test[0][1][:30]

# 어라?

Vector size: 300
[0.5782] 검색/NNG
[0.5195] 공유/NNG
[0.4650] 이용/NNG
[0.4524] 도전/NNG
[0.4519] 인정/NNG
[0.4482] 강추/NNG
[0.4480] 소개/NNG
[0.4390] 설명/NNG
[0.4382] 구입/NNG
[0.4379] 활용/NNG
First train vector:
[ 0.01600543 -0.00651022  0.01223357 -0.0164253  -0.04305169 -0.02754103
  0.00641682  0.02133358  0.00195862  0.03094834 -0.01048181  0.05453731
  0.01082994  0.01413122 -0.00510732 -0.01293624 -0.09675323  0.0216133
  0.00359899 -0.03580899  0.00970891 -0.0455177  -0.04296957  0.02304112
  0.00643211 -0.0117652  -0.02606681  0.04571986 -0.04192316  0.02901732]
First test vector:
[ 0.12037679 -0.00923613  0.17762448  0.00458726 -0.07375859  0.04501883
  0.06845441 -0.07002268  0.05921951  0.04273745 -0.04647292  0.09041403
 -0.07882628  0.04966812 -0.0148804  -0.00049853  0.03969754 -0.01020798
  0.02115483  0.06140548  0.11471982  0.035046   -0.02384861  0.06700817
  0.11634289 -0.0284129   0.0129131   0.01837295  0.00891465 -0.17598158]


* 댓글 코퍼스는 [comment_corpus.ipynb](comment_corpus.ipynb)에서..

In [ ]:
### 평가를 해봅니다.

evaluate(d2v_train, d2v_test)

# 아놔!

0.713

![](http://cdn.sketchpan.com/member/g/gkdms196/draw/1216712768375/0.png)

### 주의 사항

* gensim의 doc2vec 모델의 infer_vector 함수를 이용해 벡터를 생성하면, 같은 단어열(문서)에 대해서도 여러번 호출하면 각기 다른 벡터가 나옵니다.

* 이는 문서 벡터 추론 과정에서 랜덤하게 초기화하는 과정이 있기 때문인데, 문제는 결과로 나오는 벡터의 차이가 꽤 큽니다.

In [ ]:
import gensim

d2v_model = gensim.models.Doc2Vec.load('ratings_train.d2v')

print d2v_model.infer_vector(train_corpus[0][1])[:6]
print d2v_model.infer_vector(train_corpus[0][1])[:6]
print d2v_model.infer_vector(train_corpus[0][1])[:6]
print d2v_model.infer_vector(train_corpus[0][1])[:6]
print d2v_model.infer_vector(train_corpus[0][1])[:6]

[ 0.03026047 -0.03027695  0.03862283  0.04425745 -0.01662277 -0.04474263]
[ 0.02785604 -0.03551881  0.02772081  0.03890218 -0.02765364 -0.0447976 ]
[ 0.02609842 -0.03922388  0.03076469  0.05998807 -0.0312253  -0.02774168]
[ 0.02499082 -0.02231631  0.03571737  0.02024948 -0.01098481 -0.02605485]
[ 0.0153047  -0.03797822 -0.00724502  0.03604603 -0.0209945  -0.00081594]


* 따라서 좀더 안정적인 문서 벡터를 얻기 위해서는 1000번 정도 추론을 한 다음 평균을 내는 것이 좋을 것 같습니다.

In [ ]:
import numpy as npy

_ITER_NUM = 1000

def infer_vector(model, words):
    """
    doc2vec 모델과 단어열(문서)로부터 벡터를 추론하는 함수
    :param  model:  doc2vec 모델
    :param  words:  단어열(문서)
    :return:        추론한 벡터
    """
    vec_avg = npy.zeros([model.vector_size,])
    for _ in range(_ITER_NUM):
        vec_avg += model.infer_vector(words)
    return vec_avg / _ITER_NUM

print infer_vector(d2v_model, train_corpus[0][1])[:6]
print infer_vector(d2v_model, train_corpus[0][1])[:6]
print infer_vector(d2v_model, train_corpus[0][1])[:6]
print infer_vector(d2v_model, train_corpus[0][1])[:6]
print infer_vector(d2v_model, train_corpus[0][1])[:6]

# 안그래도 느려 죽겠는데, 1000배나 더 느려진다니.. 쳇!

[ 0.02751252 -0.0299906   0.02400205  0.04046407 -0.01994828 -0.02474876]
[ 0.02813463 -0.03087953  0.0240797   0.04082712 -0.02001685 -0.02608088]
[ 0.02803052 -0.03055546  0.0241527   0.04122491 -0.01994756 -0.02526346]
[ 0.02831996 -0.03050167  0.02429034  0.04082855 -0.01990961 -0.02531239]
[ 0.02760822 -0.03063143  0.02334326  0.04009719 -0.01998218 -0.02587145]


![](http://pds11.egloos.com/pds/200812/08/79/f0066979_493c78f018832.jpg)

5. 그때 그 방법 (word2vec)
----

* 에라 모르겠다 doc2vec이고 나발이고 word2vec을 날로 한번 써보자!

In [ ]:
### word2vec tutorial에서 사용했던 그때 그 벡터 더하기 신공!

import gensim
import numpy as npy

def to_w2v_vector(model, features):
    """
    word2vec 모델을 이용하여 모든 자질을 더해서 하나의 벡터로 표현하는 함수
    @param  model     word2vec 모델
    @param  features  자질 목록
    @return           벡터
    """
    vec_sum = npy.zeros([model.vector_size,])    # word2vec 모델의 벡터 크기 만큼 벡터를 0으로 초기화 합니다.
    for feature in set(features):
        try:
            vec_sum += model[feature]
        except KeyError:
            pass    # Out-Of-Vocabulary 자질들은 그냥 넘어갑니다.
    return vec_sum

# train 코퍼스로 만든 word2vec 모델을 읽어들입니다.
# word2vec 학습은 요렇게: ./word2vec -train ratings_train.tag -output ratings_train.w2v -cbow 1 -size 200 -window 8 -negative 25 -hs 0 -sample 1e-4 -threads 4 -binary 1 -iter 15
w2v_model = gensim.models.Word2Vec.load_word2vec_format('ratings_train.w2v', binary=True)
print 'Vector size:', w2v_model.vector_size

# train 코퍼스를 전부 벡터로 표현합니다.
w2v_train = [(rating, to_w2v_vector(w2v_model, features)) for _, features, rating in train_corpus]
print 'First train vector:'
print w2v_train[0][1][:30]

# test 코퍼스를 전부 벡터로 표현합니다.
w2v_test = [(rating, to_w2v_vector(w2v_model, features)) for _, features, rating in test_corpus]
print 'First train vector:'
print w2v_train[0][1][:30]

Vector size: 200
First train vector:
[ 0.00338062  0.14115795 -0.11022999 -0.13526215  0.31291915  0.38900751
 -0.24651022  0.279004    0.13029095  0.41423567  0.50058746 -0.29609317
  0.09810474 -0.13254156 -0.13695164 -0.2279217   0.13970647  0.09488842
 -0.16022483 -0.33515607  0.08950662  0.12349108  0.04820246  0.12442232
 -0.08895652  0.22333013  0.02346406  0.27419811 -0.42028924 -0.68062997]
First train vector:
[ 0.00338062  0.14115795 -0.11022999 -0.13526215  0.31291915  0.38900751
 -0.24651022  0.279004    0.13029095  0.41423567  0.50058746 -0.29609317
  0.09810474 -0.13254156 -0.13695164 -0.2279217   0.13970647  0.09488842
 -0.16022483 -0.33515607  0.08950662  0.12349108  0.04820246  0.12442232
 -0.08895652  0.22333013  0.02346406  0.27419811 -0.42028924 -0.68062997]


In [ ]:
### 평가를 해봅니다.

evaluate(w2v_train, w2v_test)

# 틀렸어.. ㅠ.ㅠ

0.82626

![](http://cfile30.uf.tistory.com/image/27500C33526F5E5118BFC1)